Tutorial 4: 3D motion estimation
Script for Optical Flow
Practical Course: Learning For Self-Driving Cars and Intelligent Systems
Summer Semester 2021
Technical University of Munich
Tutor: Qadeer Khan
https://vision.in.tum.de/teaching/ss2021/intellisys_ss2021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import os
import copy
import glob
import cv2
FOLDER = "/storage/group/intellisys/datasets/carla/3d_motion/stationary/episode_000/"

In [2]:
def read_rgb(rgb_file):
    rgb = io.imread(rgb_file)
    return rgb

def read_depth(depth_file):
    depth = io.imread(depth_file)
#     Reference: https://carla.readthedocs.io/en/stable/cameras_and_sensors/#camera-depth-map
    depth = depth[:, :, 0] * 1.0 + depth[:, :, 1] * 256.0 + depth[:, :, 2] * (256.0 * 256)
    depth = depth * (1/ (256 * 256 * 256 - 1))
    return depth

Dense Optical Flow: Gunner Farneback’s algorithm
Reference: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html#dense-optical-flow-in-opencv

In [3]:
cam_folder = os.path.join(FOLDER, "CameraRGB0")
all_images = sorted(glob.glob(cam_folder + '/**/*.png', recursive=True))
num_images = len(all_images)
prvs = read_rgb(all_images[0])
hsv = np.zeros_like(prvs)
hsv[...,1] = 255
prvs = cv2.cvtColor(prvs,cv2.COLOR_BGR2GRAY)

for i in range(1,num_images):
    frame2 = read_rgb(all_images[i])
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None,pyr_scale=0.5, levels=3, winsize=15,
                                      iterations=10, poly_n=5, poly_sigma=1.2,
                                      flags=0)
                                        
                                        
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    cv2.imshow('frame2',np.concatenate((frame2[...,::-1], rgb), axis=1))
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png',frame2)
        cv2.imwrite('opticalhsv.png',rgb)
    prvs = next

cv2.destroyAllWindows()

Lucas-Kanade Method
Reference: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_video/py_lucas_kanade/py_lucas_kanade.html#lucas-kanade-optical-flow-in-opencv
        

In [4]:
cam_folder = os.path.join(FOLDER, "CameraRGB0")
all_images = sorted(glob.glob(cam_folder + '/**/*.png', recursive=True))
num_images = len(all_images)
prvs = read_rgb(all_images[0])
hsv = np.zeros_like(prvs)
hsv[...,1] = 255


# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

old_gray = cv2.cvtColor(prvs, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(prvs)

for i in range(1,num_images):
    frame = read_rgb(all_images[i])
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img[...,::-1])
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()


<ipython-input-4-5e2f6117a4fb>:44: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
<ipython-input-4-5e2f6117a4fb>:45: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
